In [1]:
import argparse
import sys
sys.path.append("..")
import os

In [2]:
#Data Preprocessing for TB fasta files

In [3]:
#res_file_location = os.path.join('/home/mashjunior/Downloads/', 'rif_resis_rpob', '*.fasta')
#susc_file_location = os.path.join('/home/mashjunior/Downloads/', 'rif_susc_rpob', '*.fasta')
#print(res_file_location)
#print(susc_file_location)

In [4]:
#import glob
#res_filenames = glob.glob(res_file_location)
#print(res_filenames)
#susc_filenames = glob.glob(susc_file_location)
#print(susc_filenames)

In [5]:

#def convert(s):
#  
#    # initialization of string to ""
#    new = ""
  

# traverse in the string 
#    for x in s:
#        new += x 
#  
    # return string 
#    return new

In [6]:
#def build_kmers(sequence, ksize):
#    kmers = []
#    n_kmers = len(sequence) - ksize + 1

#    for i in range(n_kmers):
#        kmer = sequence[i:i + ksize]
#        kmers.append(kmer)

#    return kmers

In [7]:
#res_list = []
#for f in res_filenames:
#    rif_resis_rpob = open(f,'r')
#    res_data = rif_resis_rpob.readlines()
#    res_data = res_data[1:]
#    res_data = [x.strip() for x in res_data]
#    res_data = convert(res_data)
#    res_data = ' '.join(build_kmers(res_data, 6))
#    res_list.append(res_data)

In [8]:
#susc_list = []
#for f in susc_filenames:
#    rif_susc_rpob = open(f,'r')
#    susc_data = rif_susc_rpob.readlines()
#    susc_data = susc_data[1:]
#    susc_data = [x.strip() for x in susc_data]
#    susc_data = convert(susc_data)
#    susc_data = ' '.join(build_kmers(susc_data, 6))
#    susc_list.append(susc_data)

In [9]:
#a_list = ["abc", "def", "ghi"]
#textfile = open("TB_contig.tsv", "w")
#textfile.write('TEXT' + ',' + 'LABEL' + "\n")
#for res, susc in zip(susc_list, res_list):
#    textfile.write(res + ',' '1' + "\n")
#    textfile.write(susc + ',' '0' + "\n")
#textfile.close()

In [10]:
#file = open("/home/mashjunior/Desktop/FactorEmbeddings/embedding_eperiments/tt_embeddings/sentiment/TB_contig.tsv", "r")
#lines = file.readlines()

In [11]:
#for line in lines:
 #   if line == " ":
 #       print("Empty Line")

In [12]:
import torch 
#print(torch.__version__)

In [13]:
parser = argparse.ArgumentParser()
parser.add_argument(
    '--embedding', 
    default='tt',
    choices=['tt', 'tr', 'full'],
    type=str)
parser.add_argument('--ranks', type=int, default=8)
parser.add_argument('--d', type=int, default=3)
parser.add_argument('--embed_dim', type=int, default=64)
parser.add_argument('--voc_dim', default=250, type=int)
parser.add_argument('--lr', default=5e-4)
parser.add_argument('--gpu', default='', type=str)
parser.add_argument('--hidden_dim', default=128, type=int)
parser.add_argument('--n_epochs',  default=10, type=int)
parser.add_argument('--fout',  default="logdir/", type=str)
parser.add_argument('--dropout', default=0.5, type=float)
parser.add_argument(
    '--dataset',
    default='TB',
    type=str)
args = parser.parse_args('')

In [14]:
if args.embedding == 'tt':
    tt = "tt"
elif args.embedding == 'tr':
    tt = 'tr'
else:             
    tt = "full"

In [15]:
model_name = f"{args.dataset}-dim_{args.embed_dim}-d_{args.d}-ranks_{args.ranks}-{tt}"

In [16]:
import os
os.environ['CUDA_DEVICE_ORDER']='PCI_BUS_ID'
os.environ['CUDA_VISIBLE_DEVICES']=args.gpu
import utils
import torch
import numpy as np
import torch.nn.functional as F
import torch.nn as nn
import t3nsor as t3
from torchtext import data
from torchtext import datasets
import torch.optim as optim
from models import LSTM_Classifier
#from utils import binary_accuracy, train, evaluate
import pickle
import random
import spacy
from spacy.cli.download import download
random.seed(42)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
nlp = spacy.load("en_core_web_sm")

/home/mashjunior/anaconda3/envs/james_tensor/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/mashjunior/anaconda3/envs/james_tensor/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/mashjunior/anaconda3/envs/james_tensor/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/mashjunior

In [17]:
print("Building the dataset")
data_path = '/home/mashjunior/Desktop/FactorEmbeddings/embedding_eperiments/tt_embeddings/sentiment/TB_contig.tsv'
TEXT = data.Field(tokenize='spacy', fix_length=100)
LABEL = data.LabelField(dtype=torch.float)
fields = [('text', TEXT),('label', LABEL)]
training_data = data.TabularDataset(
    path=data_path,
    format='csv',
    fields=fields,
    skip_header=True,
)
print("done")

Building the dataset
done


In [18]:
train_data, val_data = training_data.split(split_ratio=0.7, random_state=random.seed(42))

In [19]:
val_data, test_data = val_data.split(split_ratio=0.2, random_state=random.seed(42))

In [20]:
device

device(type='cpu')

In [21]:
OUTPUT_DIM = 1
#train_data, test_ = datasets.IMDB.splits(TEXT, LABEL)
#test_list = list(test_)
#random.shuffle(test_list)
#test_data_ = test_list[:12500]
#val_data_ = test_list[12500:]
#train_data = data.dataset.Dataset(train_data, fields=[('text', TEXT), ('label', LABEL)])
valid_data = data.dataset.Dataset(val_data, fields=fields)
test_data = data.dataset.Dataset(test_data, fields=fields)

In [22]:
def sort_key(ex):
    return len(ex.text)

In [23]:
TEXT.build_vocab(train_data, max_size=args.voc_dim - 2)
LABEL.build_vocab(train_data)

BATCH_SIZE = 64

In [24]:
train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data),
    batch_size=BATCH_SIZE,
    device=device)

valid_iterator.sort_key = sort_key
test_iterator.sort_key = sort_key

In [25]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = args.embed_dim
HIDDEN_DIM = args.hidden_dim
N_LAYERS = 2
BIDIRECTIONAL = True
DROPOUT = args.dropout

In [26]:
actual_vocab_size = len(TEXT.vocab.stoi)

In [27]:
actual_vocab_size

250

In [28]:
lstm_model = LSTM_Classifier(embedding_dim=EMBEDDING_DIM,
                             hidden_dim=HIDDEN_DIM,
                             output_dim=OUTPUT_DIM,
                             n_layers=N_LAYERS,
                             bidirectional=BIDIRECTIONAL,
                             dropout=DROPOUT)

In [29]:
if args.embedding == 'tt':
        embed_model = t3.TTEmbedding(
            voc_size=INPUT_DIM,
            emb_size=EMBEDDING_DIM,
            auto_shapes=True,
            auto_shape_mode='mixed',
            d=args.d,
            tt_rank=args.ranks,
            padding_idx=1
        )
        compression_rate = INPUT_DIM * EMBEDDING_DIM / embed_model.tt_matrix.dof
elif args.embedding == 'tr':
        embed_model = t3.TREmbedding(
            voc_size=INPUT_DIM,
            emb_size=EMBEDDING_DIM,
            auto_shapes=True,
            auto_shape_mode='mixed',
            d=args.d,
            tr_rank=args.ranks,
            padding_idx=1
        )
        compression_rate = INPUT_DIM * EMBEDDING_DIM / embed_model.tr_matrix.dof
else:
    embed_model = nn.Embedding(
        num_embeddings=INPUT_DIM,
        embedding_dim=EMBEDDING_DIM
    )
    compression_rate = 1.0


def cross_entropy_loss(logits, target):
    labels = target.type(torch.LongTensor).to(logits.device)
    return nn.CrossEntropyLoss()(logits, labels)

In [30]:
model = nn.Sequential(embed_model, lstm_model)

In [31]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    if len(preds.shape) == 1:
        rounded_preds = torch.round(torch.sigmoid(preds))
    else:
        rounded_preds = preds.argmax(1)
    correct = (rounded_preds == y).float() #convert into float for division
    acc = correct.sum()/len(correct)
    return acc

In [32]:
def train(model, iterator, optimizer, criterion):

    epoch_loss = 0
    epoch_acc = 0
    total_len = 0

    model.train()
    
    if isinstance(criterion, nn.CrossEntropyLoss):
        dtype = torch.LongTensor
    elif isinstance(criterion, nn.BCEWithLogitsLoss):
        dtype = torch.FloatTensor

    for i, batch in enumerate(iterator):

        optimizer.zero_grad()
        device = batch.text.device
        labels = batch.label.type(dtype).to(device)
        predictions = model(batch.text).squeeze(1)
        loss = criterion(predictions, labels)
        acc = binary_accuracy(predictions, labels)
        loss.backward()
        optimizer.step()

        B = batch.label.shape[0]

        epoch_loss += B * loss.item()
        epoch_acc += B * acc.item()

        total_len += B


        if i > len(iterator):
            break

    return epoch_loss / total_len, epoch_acc / total_len

In [33]:
def evaluate(model, iterator, criterion):

    epoch_loss = 0
    epoch_acc = 0
    total_len = 0

    model.eval()
    
    if isinstance(criterion, nn.CrossEntropyLoss):
        dtype = torch.LongTensor
    elif isinstance(criterion, nn.BCEWithLogitsLoss):
        dtype = torch.FloatTensor

    with torch.no_grad():

        for i, batch in enumerate(iterator):
            
            device = batch.text.device
            labels = batch.label.type(dtype).to(device)
            predictions = model(batch.text).squeeze(1)

            loss = criterion(predictions, labels)

            acc = binary_accuracy(predictions, labels)
            B = batch.label.shape[0]

            epoch_loss += B * loss.item()
            epoch_acc += B * acc.item()
            total_len += B

            if i > len(iterator):
                break

    return epoch_loss / total_len, epoch_acc / total_len

In [34]:
if args.dataset == 'TB':
    criterion = nn.BCEWithLogitsLoss()
    criterion = criterion.to(device)
#elif args.dataset[:3] == 'sst':
#    criterion = nn.CrossEntropyLoss()
    #criterion = criterion.to(device)
else:
    raise NotImplementedError

#criterion = nn.BCEWithLogitsLoss()
#criterion = criterion.to(device)

In [35]:
model = model.to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)
print(model)
N_EPOCHS = args.n_epochs

Sequential(
  (0): TTEmbedding(
    (parameters): ParameterList(
        (0): Parameter containing: [torch.FloatTensor of size 1x5x4x8]
        (1): Parameter containing: [torch.FloatTensor of size 8x6x4x8]
        (2): Parameter containing: [torch.FloatTensor of size 8x10x4x1]
    )
  )
  (1): LSTM_Classifier(
    (rnn): LSTM(64, 128, num_layers=2, dropout=0.5, bidirectional=True)
    (fc): Linear(in_features=256, out_features=1, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)


In [41]:
1*5*4*8

160

In [36]:
log = {
    'compression_rate':compression_rate,
    'train_loss':[], 'test_loss':[], 'valid_loss':[],
    'train_acc':[], 'test_acc':[], 'valid_acc':[]}
best_result = {
    "epoch": 0, "train_acc": 0, "valid_acc": 0, "train_acc": 0}

In [38]:
for epoch in range(N_EPOCHS):

    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    test_loss, test_acc = evaluate(model, test_iterator, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)

    log['train_loss'].append(train_loss)
    log['test_loss'].append(test_loss)
    log['train_acc'].append(train_acc)
    log['test_acc'].append(test_acc)
    log['valid_acc'].append(valid_acc)
    log['valid_loss'].append(valid_loss)

    if best_result["valid_acc"] < valid_acc:
        best_result["epoch"] = epoch
        best_result["train_acc"] = train_acc
        best_result["valid_acc"] = valid_acc
        best_result["test_acc"] = test_acc

    #if args.fout is not None:
    #    with open(args.fout+f"{model_name}-best.pkl", 'wb') as f:
    #        pickle.dump(best_result, f)
    print(f'| Epoch: {epoch+1:.2f} | Train Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}% | Val. Loss: {valid_loss:.3f} | Val. Acc: {valid_acc*100:.2f}% | Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}% |')
    print ("TEST ACCURACY:", np.round(best_result["test_acc"] * 100, 2))

| Epoch: 1.00 | Train Loss: 0.694 | Train Acc: 50.36% | Val. Loss: 0.694 | Val. Acc: 45.83% | Test Loss: 0.692 | Test Acc: 48.96% |
TEST ACCURACY: 48.96
| Epoch: 2.00 | Train Loss: 0.690 | Train Acc: 50.71% | Val. Loss: 0.694 | Val. Acc: 45.83% | Test Loss: 0.691 | Test Acc: 48.96% |
TEST ACCURACY: 48.96
| Epoch: 3.00 | Train Loss: 0.690 | Train Acc: 55.71% | Val. Loss: 0.692 | Val. Acc: 50.00% | Test Loss: 0.690 | Test Acc: 51.04% |
TEST ACCURACY: 51.04
| Epoch: 4.00 | Train Loss: 0.686 | Train Acc: 60.36% | Val. Loss: 0.687 | Val. Acc: 66.67% | Test Loss: 0.685 | Test Acc: 73.96% |
TEST ACCURACY: 73.96
| Epoch: 5.00 | Train Loss: 0.682 | Train Acc: 64.29% | Val. Loss: 0.681 | Val. Acc: 66.67% | Test Loss: 0.678 | Test Acc: 76.04% |
TEST ACCURACY: 73.96
| Epoch: 6.00 | Train Loss: 0.682 | Train Acc: 57.14% | Val. Loss: 0.674 | Val. Acc: 54.17% | Test Loss: 0.665 | Test Acc: 54.17% |
TEST ACCURACY: 73.96
| Epoch: 7.00 | Train Loss: 0.661 | Train Acc: 62.50% | Val. Loss: 0.650 | Val. Ac

In [39]:
log

{'compression_rate': 7.936507936507937,
 'train_loss': [0.6938424331801278,
  0.6938001275062561,
  0.6901356066976275,
  0.6901510562215533,
  0.6858684931482587,
  0.681563379083361,
  0.6820732917104448,
  0.6612920267241341,
  0.6273076891899109,
  0.5302114622933524,
  0.4490554256098611],
 'test_loss': [0.695839007695516,
  0.6924671729405721,
  0.6914886832237244,
  0.6897042393684387,
  0.685492992401123,
  0.6779112219810486,
  0.6650245189666748,
  0.6427273154258728,
  0.5886634588241577,
  0.5051289598147074,
  0.3303088943163554],
 'valid_loss': [0.6998911499977112,
  0.6943073272705078,
  0.6936776638031006,
  0.6923295855522156,
  0.6874141693115234,
  0.6806931495666504,
  0.6743363738059998,
  0.6495236754417419,
  0.6231448650360107,
  0.5936792492866516,
  0.3901023864746094],
 'train_acc': [0.507142858845847,
  0.5035714294229235,
  0.5071428571428571,
  0.5571428571428572,
  0.6035714302744184,
  0.6428571428571429,
  0.5714285722800664,
  0.6249999982970101,
  0.6

In [40]:
best_result

{'epoch': 9,
 'train_acc': 0.7857142857142857,
 'valid_acc': 1.0,
 'test_acc': 0.9895833333333334}